In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn

## 데이터 로드 및 train test로 분할

In [72]:
# df_kookmin = pd.read_csv('/Users/lhs2/Desktop/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kookmin.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])
# df_kyunghyan = pd.read_csv('/Users/lhs2/Desktop/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kyunghyan.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])

df_kookmin = pd.read_csv('D:/Documents/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kookmin.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])
df_kyunghyan = pd.read_csv('D:/Documents/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kyunghyan.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])


In [73]:
df = pd.concat([df_kookmin, df_kyunghyan]).drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','links'])
# df = df.set_index(['dates'])

In [74]:
df

,titles,dates,contents
0,"[IPO 앞둔 오아시스마켓, 매장 간판서 ‘생협’ 뺀다]",2022-04-27 04:06,[\n\n우리소비자생활협동조합의 위탁판매점이라는 표시를 한 오아시스 매장 모습. 오...
1,"[10년간 이어진 ‘생협’ 명칭 논란…오아시스, 간판에서 ‘생협’ 뗀다]",2022-04-26 17:20,"[\n\n우리생협 오아시스 신촌점(왼쪽)과 오아시스 주엽역점. 우리생협, 오아시스 ..."
2,[1시간 내 장보기 서비스 ‘요마트’ 재가동],2022-05-18 04:06,[\n\nGS리테일과 요기요가 17일 전국 즉시 장보기 서비스 ‘요마트’를 오픈했다...
3,"[요기요+GS리테일, ‘요마트’ 부활… ‘퀵커머스’ 치열한 경쟁]",2022-05-17 17:11,[\n\nGS리테일과 요기요가 17일 전국 즉시 장보기 서비스 ‘요마트’를 오픈했다...
4,[엔데믹 임박… 잘나가던 새벽배송 ‘탈진’],2022-04-18 04:06,[\n\nBGF는 헬로네이처에서 운영하던 새벽배송 서비스를 다음 달 말에 종료한다고...
5,"[“출혈경쟁 못 견뎌” 롯데온, BGF 새벽배송 접는다]",2022-04-17 17:22,[\n\nBGF가 운영하는 새벽배송 업체 '헬로네이처'. BGF 제공롯데온에 이어 ...
6,[2차전지 소재 유망기업 대구 유치…1530억원 투자],2022-03-27 13:18,[\n\n대구국가산단 조감도. 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 ...
7,"[SSG닷컴, “내년 증권시장 상장 목표”…대표 주간사 선정]",2021-10-27 16:23,[\n\nSSG닷컴이 기업공개(IPO)를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을...
8,"[마켓컬리, SSG, 오아시스마켓 물류센터에 산업재해 ‘급증’]",2022.08.15 10:22,[물류창고의 좁은 통로에서는 등 뒤로 지게차가 지나다녔다. 지게차의 지게 발을 피하...
9,[IPO 혹한기 속 누가 웃을까…컬리·11번가·오아시스 상장 시동],2022.08.29 16:26,[주요 e커머스 플랫폼이 기업공개(IPO) 레이스에 시동을 걸었다. 컬리를 시작으로...


In [8]:
for i in range(28):
    df.dates[i] =df.dates[i][:10]

In [9]:
df['dates'] = df['dates'].str.replace('.','-')
df['dates'] = df['dates'].str.replace(' ', '')

C:\Users\hslio\AppData\Local\Temp\ipykernel_17360\3561615856.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['dates'] = df['dates'].str.replace('.','-')


In [10]:
df = df.sort_values(by=['dates'], axis=0).reset_index().drop(columns=['index'])#.drop(columns=['level_0'])
df

,titles,dates,contents
0,"[SSG닷컴, “내년 증권시장 상장 목표”…대표 주간사 선정]",2021-10-27,[\n\nSSG닷컴이 기업공개(IPO)를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을...
1,[새벽배송 1호 상장사 어디?…SSG닷컴·컬리·오아시스 IPO 경쟁],2021-10-31,[‘새벽배송’을 대표하는 기업들이 최근 경쟁적으로 상장 절차에 돌입하면서 ‘1호 상...
2,[새벽배송 ‘1호 상장사’ 주인공은 누가 될까?],2021-10-31,[마켓컬리·SSG닷컴·오아시스마켓내년 국내 증시 상장을 목표로대표 주간사 선정 잇따...
3,[내년 IPO 규모 30조원대...역대급을 넘어서는 역대급],2021-12-29,[올해 역대 최대를 기록했던 증권 시장의 기업공개(IPO) 규모가 2022년에는 더...
4,[예상 가치 수조원대 기업 ‘줄줄이’…내년 IPO ‘신기록 행진’ 이어지나],2021-12-29,[1월 상장 추진 LG에너지솔루션시총 목표 70조 ‘최대어’ 1순위현대엔지니어링 최...
5,"[네이버·홈쇼핑 등 새벽배송 진출 러쉬…""온라인 장보기 접수""]",2022-03-08,[온라인 장보기 일상화로 신선식품 새벽배송 시장에 신규 사업자들이 잇따라 진출하고 ...
6,[후발주자 잇단 참전…치열해진 새벽배송],2022-03-08,"[네이버, SSG닷컴과 연계해 진출온라인몰인 G마켓·인터파크에CJ온스타일 등 홈쇼핑..."
7,[2차전지 소재 유망기업 대구 유치…1530억원 투자],2022-03-27,[\n\n대구국가산단 조감도. 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 ...
8,[직원 스마트폰 앱 ‘오아시스 루트’…집화·포장·배송 최적화 ‘흑자 루트’],2022-04-10,"[새벽 식품 배송업계서 유일 이익오아시스마켓 물류센터 가보니, 2018년 진출…작년..."
9,"[“출혈경쟁 못 견뎌” 롯데온, BGF 새벽배송 접는다]",2022-04-17,[\n\nBGF가 운영하는 새벽배송 업체 '헬로네이처'. BGF 제공롯데온에 이어 ...


In [11]:
# index 날짜로 변경

df = df.set_index('dates')
df

,titles,contents
dates,,
2021-10-27,"[SSG닷컴, “내년 증권시장 상장 목표”…대표 주간사 선정]",[\n\nSSG닷컴이 기업공개(IPO)를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을...
2021-10-31,[새벽배송 1호 상장사 어디?…SSG닷컴·컬리·오아시스 IPO 경쟁],[‘새벽배송’을 대표하는 기업들이 최근 경쟁적으로 상장 절차에 돌입하면서 ‘1호 상...
2021-10-31,[새벽배송 ‘1호 상장사’ 주인공은 누가 될까?],[마켓컬리·SSG닷컴·오아시스마켓내년 국내 증시 상장을 목표로대표 주간사 선정 잇따...
2021-12-29,[내년 IPO 규모 30조원대...역대급을 넘어서는 역대급],[올해 역대 최대를 기록했던 증권 시장의 기업공개(IPO) 규모가 2022년에는 더...
2021-12-29,[예상 가치 수조원대 기업 ‘줄줄이’…내년 IPO ‘신기록 행진’ 이어지나],[1월 상장 추진 LG에너지솔루션시총 목표 70조 ‘최대어’ 1순위현대엔지니어링 최...
2022-03-08,"[네이버·홈쇼핑 등 새벽배송 진출 러쉬…""온라인 장보기 접수""]",[온라인 장보기 일상화로 신선식품 새벽배송 시장에 신규 사업자들이 잇따라 진출하고 ...
2022-03-08,[후발주자 잇단 참전…치열해진 새벽배송],"[네이버, SSG닷컴과 연계해 진출온라인몰인 G마켓·인터파크에CJ온스타일 등 홈쇼핑..."
2022-03-27,[2차전지 소재 유망기업 대구 유치…1530억원 투자],[\n\n대구국가산단 조감도. 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 ...
2022-04-10,[직원 스마트폰 앱 ‘오아시스 루트’…집화·포장·배송 최적화 ‘흑자 루트’],"[새벽 식품 배송업계서 유일 이익오아시스마켓 물류센터 가보니, 2018년 진출…작년..."


## 같은 날짜 한줄에 표시

In [12]:
df = df.groupby('dates').sum()
df

,titles,contents
dates,,
2021-10-27,"[SSG닷컴, “내년 증권시장 상장 목표”…대표 주간사 선정]",[\n\nSSG닷컴이 기업공개(IPO)를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을...
2021-10-31,[새벽배송 1호 상장사 어디?…SSG닷컴·컬리·오아시스 IPO 경쟁][새벽배송 ‘1...,[‘새벽배송’을 대표하는 기업들이 최근 경쟁적으로 상장 절차에 돌입하면서 ‘1호 상...
2021-12-29,[내년 IPO 규모 30조원대...역대급을 넘어서는 역대급][예상 가치 수조원대 기...,[올해 역대 최대를 기록했던 증권 시장의 기업공개(IPO) 규모가 2022년에는 더...
2022-03-08,"[네이버·홈쇼핑 등 새벽배송 진출 러쉬…""온라인 장보기 접수""][후발주자 잇단 참전...",[온라인 장보기 일상화로 신선식품 새벽배송 시장에 신규 사업자들이 잇따라 진출하고 ...
2022-03-27,[2차전지 소재 유망기업 대구 유치…1530억원 투자],[\n\n대구국가산단 조감도. 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 ...
2022-04-10,[직원 스마트폰 앱 ‘오아시스 루트’…집화·포장·배송 최적화 ‘흑자 루트’],"[새벽 식품 배송업계서 유일 이익오아시스마켓 물류센터 가보니, 2018년 진출…작년..."
2022-04-17,"[“출혈경쟁 못 견뎌” 롯데온, BGF 새벽배송 접는다]",[\n\nBGF가 운영하는 새벽배송 업체 '헬로네이처'. BGF 제공롯데온에 이어 ...
2022-04-18,[엔데믹 임박… 잘나가던 새벽배송 ‘탈진’],[\n\nBGF는 헬로네이처에서 운영하던 새벽배송 서비스를 다음 달 말에 종료한다고...
2022-04-26,"[10년간 이어진 ‘생협’ 명칭 논란…오아시스, 간판에서 ‘생협’ 뗀다]","[\n\n우리생협 오아시스 신촌점(왼쪽)과 오아시스 주엽역점. 우리생협, 오아시스 ..."


## null 값 확인

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2021-10-27 to 2022-08-29
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   titles    19 non-null     object
 1   contents  19 non-null     object
dtypes: object(2)
memory usage: 456.0+ bytes


## 특수문자 제거 정규표현식

In [14]:
df['titles'] = df['titles'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
df['contents'] = df['contents'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')

C:\Users\hslio\AppData\Local\Temp\ipykernel_17360\1747014847.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['titles'] = df['titles'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
C:\Users\hslio\AppData\Local\Temp\ipykernel_17360\1747014847.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['contents'] = df['contents'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')


In [15]:
df

,titles,contents
dates,,
2021-10-27,SSG닷컴 내년 증권시장 상장 목표대표 주간사 선정,\n\nSSG닷컴이 기업공개IPO를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을 대표...
2021-10-31,새벽배송 1호 상장사 어디SSG닷컴컬리오아시스 IPO 경쟁새벽배송 1호 상장사 주인...,새벽배송을 대표하는 기업들이 최근 경쟁적으로 상장 절차에 돌입하면서 1호 상장사 타...
2021-12-29,내년 IPO 규모 30조원대역대급을 넘어서는 역대급예상 가치 수조원대 기업 줄줄이내...,올해 역대 최대를 기록했던 증권 시장의 기업공개IPO 규모가 2022년에는 더욱 커...
2022-03-08,네이버홈쇼핑 등 새벽배송 진출 러쉬온라인 장보기 접수후발주자 잇단 참전치열해진 새벽배송,온라인 장보기 일상화로 신선식품 새벽배송 시장에 신규 사업자들이 잇따라 진출하고 있...
2022-03-27,2차전지 소재 유망기업 대구 유치1530억원 투자,\n\n대구국가산단 조감도 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 유망...
2022-04-10,직원 스마트폰 앱 오아시스 루트집화포장배송 최적화 흑자 루트,새벽 식품 배송업계서 유일 이익오아시스마켓 물류센터 가보니 2018년 진출작년 영업...
2022-04-17,출혈경쟁 못 견뎌 롯데온 BGF 새벽배송 접는다,\n\nBGF가 운영하는 새벽배송 업체 헬로네이처 BGF 제공롯데온에 이어 BGF까...
2022-04-18,엔데믹 임박 잘나가던 새벽배송 탈진,\n\nBGF는 헬로네이처에서 운영하던 새벽배송 서비스를 다음 달 말에 종료한다고 ...
2022-04-26,10년간 이어진 생협 명칭 논란오아시스 간판에서 생협 뗀다,\n\n우리생협 오아시스 신촌점왼쪽과 오아시스 주엽역점 우리생협 오아시스 제공새벽배...


## 토큰화

In [75]:
okt = Okt()


In [52]:
# 불용어 처리 

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다',
             '\n','\n ','제목','을','로','있다','등']
stopwords

['의',
 '가',
 '이',
 '은',
 '들',
 '는',
 '좀',
 '잘',
 '걍',
 '과',
 '도',
 '를',
 '으로',
 '자',
 '에',
 '와',
 '한',
 '하다',
 '\n',
 '\n ',
 '제목',
 '을',
 '로',
 '있다',
 '등']

In [53]:
token = []
for sentence in tqdm(df['contents']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    token.append(stopwords_removed_sentence)
    

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00, 11.82it/s]


In [54]:
len(token)

19

In [79]:
print(token[:1])

[['\n\n', 'SSG', '닷', '컴', '기업', '공개', 'IPO', '위해', '미래에셋', '증권', '씨티', '그룹', '글로벌', '마켓', '증권', '대표', '주', '간사', '선정', 'SSG', '닷', '컴', '내년', '증권', '시장', '상장', '목표', 'SSG', '닷', '컴', '지난', '8월', '입찰', '제안', '서', 'RFP', '발송', '이후', '뜨겁다', '관심', '받다', '미래에셋', '증권', '씨티', '그룹', '글로벌', '마켓', '증권', '대표', '주', '간사', '선정', '하고', '외국', '계', '투자', '은행', '인', '모건스탠리', '제', '피', '모간', '체이스', '공동', '주', '간사', '참여', '돼다', '27일', '밝히다', 'SSG', '닷', '컴', '지난해', '말', '기준', '1조', '4000원', '자본', '총계', '기록', '하고', 'SSG', '닷', '컴', '올해', '상반기', '매출', '6866억원', '지난해', '상반기', '보다', '110', '증가', '상반기', '총', '거래', '액', '2조', '5806억원', '영', '업적', '296억원', '이었다', 'SSG', '닷컴', '관계자', '법인', '출범', '이렇다', '관리', '가능하다', '수준', '손익', '유지', '하고', '이다', '자본', '잠식', '상태', '매년', '수천', '억원', '적자', '기록', '대부분', '경쟁', '사', '차별', '화', '되다', '요인', '이라고', '말', 'SSG', '닷', '컴', '국내', '상장', '통해', '자금', '확보', '되다', '물류', '인프라', 'IT', '분야', '집중', '투자', '한다는', '계획', '이다', 'SSG', '닷컴', '관계자', '앞', '투명하다', '성공', '적', '인', 'IPO', '추진'

In [80]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(token)

In [81]:
print(tokenizer.word_index)

{'배송': 1, '새벽': 2, '마켓': 3, '물류': 4, '오아시스': 5, '이다': 6, '것': 7, '센터': 8, '되다': 9, '시장': 10, '에서': 11, '적': 12, '상장': 13, '하고': 14, '수': 15, '인': 16, '컬리': 17, 'ssg': 18, '기업': 19, '업체': 20, '노동자': 21, '건': 22, '생협': 23, '다': 24, '식품': 25, '서비스': 26, '대표': 27, '말': 28, '운영': 29, '온라인': 30, '사업': 31, '고': 32, '노동': 33, '닷': 34, '컴': 35, '돼다': 36, '까지': 37, '증권': 38, '마트': 39, '않다': 40, '그룹': 41, '투자': 42, 'gs': 43, '위': 44, '경쟁': 45, '대형': 46, '관계자': 47, '일': 48, '사': 49, '전': 50, '규모': 51, '업계': 52, '커머스': 53, '최대': 54, '상품': 55, '많다': 56, '리테일': 57, '지난해': 58, '1': 59, '매장': 60, '코로나': 61, '안전': 62, 'ipo': 63, '없다': 64, '쿠팡': 65, '받다': 66, '올해': 67, '따르다': 68, '전국': 69, '신선': 70, '회장': 71, '플랫폼': 72, '오프라인': 73, '산재': 74, '불량': 75, '위해': 76, '곳': 77, '대': 78, '성': 79, '아니다': 80, '주': 81, '내년': 82, '지난': 83, '닷컴': 84, '이라고': 85, '3': 86, '중': 87, '대한': 88, '네이버': 89, '우리': 90, '19': 91, '보다': 92, '국내': 93, '나서다': 94, '가다': 95, '요': 96, '화재': 97, '흑자': 98, '주문': 99, '롯데': 100, '개선': 101

In [82]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 2528
등장 빈도가 2번 이하인 희귀 단어의 수: 1451
단어 집합에서 희귀 단어의 비율: 57.39715189873418
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 15.755530558680164


###### from collections import Counter

In [56]:
# 한 날짜? 한 행당 횟수 추출

for i in range(19):
    vocab = Counter(token[i])
    print(vocab)

Counter({'SSG': 8, '증권': 7, '닷': 6, '컴': 6, '주': 5, '간사': 5, '상장': 5, '위해': 4, '마켓': 4, '선정': 4, '하고': 4, '투자': 4, '추진': 4, 'IPO': 3, '대표': 3, '말': 3, '상반기': 3, '이다': 3, '국내': 3, '미래에셋': 2, '씨티': 2, '그룹': 2, '글로벌': 2, '내년': 2, '인': 2, '지난해': 2, '자본': 2, '기록': 2, '닷컴': 2, '관계자': 2, '되다': 2, '이라고': 2, '계획': 2, '컬리': 2, '재': 2, '\n\n': 1, '기업': 1, '공개': 1, '시장': 1, '목표': 1, '지난': 1, '8월': 1, '입찰': 1, '제안': 1, '서': 1, 'RFP': 1, '발송': 1, '이후': 1, '뜨겁다': 1, '관심': 1, '받다': 1, '외국': 1, '계': 1, '은행': 1, '모건스탠리': 1, '제': 1, '피': 1, '모간': 1, '체이스': 1, '공동': 1, '참여': 1, '돼다': 1, '27일': 1, '밝히다': 1, '기준': 1, '1조': 1, '4000원': 1, '총계': 1, '올해': 1, '매출': 1, '6866억원': 1, '보다': 1, '110': 1, '증가': 1, '총': 1, '거래': 1, '액': 1, '2조': 1, '5806억원': 1, '영': 1, '업적': 1, '296억원': 1, '이었다': 1, '법인': 1, '출범': 1, '이렇다': 1, '관리': 1, '가능하다': 1, '수준': 1, '손익': 1, '유지': 1, '잠식': 1, '상태': 1, '매년': 1, '수천': 1, '억원': 1, '적자': 1, '대부분': 1, '경쟁': 1, '사': 1, '차별': 1, '화': 1, '요인': 1, '통해': 1, '자금': 1, '확보': 1, '물류': 1, '인프라

In [57]:
# 전체 중 빈도수 출력

all_word_list = sum(token, [])
vocab = Counter(all_word_list)
print(vocab)

Counter({'배송': 248, '새벽': 177, '마켓': 129, '물류': 121, '오아시스': 119, '이다': 112, '것': 102, '센터': 100, '되다': 98, '시장': 96, '에서': 93, '적': 85, '상장': 84, '하고': 76, '수': 71, '인': 67, '컬리': 67, 'SSG': 66, '기업': 65, '업체': 60, '노동자': 59, '건': 56, '생협': 51, '다': 50, '식품': 50, '서비스': 50, '대표': 49, '말': 49, '운영': 49, '온라인': 48, '사업': 47, '고': 46, '노동': 45, '닷': 43, '컴': 43, '돼다': 43, '까지': 41, '증권': 40, '마트': 40, '않다': 39, '그룹': 38, '투자': 38, 'GS': 37, '위': 36, '경쟁': 34, '대형': 34, '관계자': 33, '일': 33, '사': 32, '전': 32, '규모': 32, '업계': 31, '커머스': 31, '최대': 31, '상품': 31, '많다': 31, '리테일': 30, '지난해': 29, '1': 29, '매장': 29, '코로나': 29, '안전': 29, 'IPO': 28, '없다': 28, '쿠팡': 28, '받다': 27, '올해': 27, '따르다': 27, '전국': 27, '신선': 27, '회장': 27, '플랫폼': 26, '오프라인': 26, '산재': 26, '불량': 26, '위해': 25, '곳': 25, '대': 25, '성': 25, '아니다': 25, '주': 24, '내년': 24, '지난': 24, '닷컴': 24, '이라고': 24, '3': 24, '중': 24, '대한': 24, '네이버': 24, '우리': 24, '19': 24, '보다': 23, '국내': 23, '나서다': 23, '가다': 23, '요': 23, '화재': 23, '흑자': 22, '주문':

In [58]:
word_list = list(vocab.keys())
word_list

['\n\n',
 'SSG',
 '닷',
 '컴',
 '기업',
 '공개',
 'IPO',
 '위해',
 '미래에셋',
 '증권',
 '씨티',
 '그룹',
 '글로벌',
 '마켓',
 '대표',
 '주',
 '간사',
 '선정',
 '내년',
 '시장',
 '상장',
 '목표',
 '지난',
 '8월',
 '입찰',
 '제안',
 '서',
 'RFP',
 '발송',
 '이후',
 '뜨겁다',
 '관심',
 '받다',
 '하고',
 '외국',
 '계',
 '투자',
 '은행',
 '인',
 '모건스탠리',
 '제',
 '피',
 '모간',
 '체이스',
 '공동',
 '참여',
 '돼다',
 '27일',
 '밝히다',
 '지난해',
 '말',
 '기준',
 '1조',
 '4000원',
 '자본',
 '총계',
 '기록',
 '올해',
 '상반기',
 '매출',
 '6866억원',
 '보다',
 '110',
 '증가',
 '총',
 '거래',
 '액',
 '2조',
 '5806억원',
 '영',
 '업적',
 '296억원',
 '이었다',
 '닷컴',
 '관계자',
 '법인',
 '출범',
 '이렇다',
 '관리',
 '가능하다',
 '수준',
 '손익',
 '유지',
 '이다',
 '잠식',
 '상태',
 '매년',
 '수천',
 '억원',
 '적자',
 '대부분',
 '경쟁',
 '사',
 '차별',
 '화',
 '되다',
 '요인',
 '이라고',
 '국내',
 '통해',
 '자금',
 '확보',
 '물류',
 '인프라',
 'IT',
 '분야',
 '집중',
 '한다는',
 '계획',
 '앞',
 '투명하다',
 '성공',
 '적',
 '추진',
 '주요한',
 '사안',
 '적다',
 '공식',
 '커뮤니케이션',
 '지속',
 '것',
 '이커머스',
 '업계',
 '최근',
 '증시',
 '적극',
 '컬리',
 '운영',
 '나서다',
 '오아시스',
 'NH',
 '한국',
 '11',
 '번가',
 '늦다',
 '2023년',
 '까지는',
 

In [59]:
word_rank = list(vocab.values())
word_rank

[8,
 66,
 43,
 43,
 65,
 10,
 28,
 25,
 4,
 40,
 4,
 38,
 7,
 129,
 49,
 24,
 14,
 11,
 24,
 96,
 84,
 13,
 24,
 4,
 1,
 1,
 13,
 1,
 1,
 10,
 1,
 6,
 27,
 76,
 1,
 15,
 38,
 1,
 67,
 1,
 20,
 1,
 3,
 1,
 1,
 4,
 43,
 2,
 16,
 29,
 49,
 20,
 8,
 1,
 11,
 5,
 17,
 27,
 10,
 11,
 1,
 23,
 1,
 11,
 10,
 7,
 6,
 8,
 1,
 5,
 1,
 1,
 1,
 24,
 33,
 3,
 5,
 5,
 11,
 15,
 8,
 6,
 7,
 112,
 3,
 9,
 5,
 4,
 4,
 12,
 15,
 34,
 32,
 8,
 14,
 98,
 7,
 24,
 23,
 20,
 11,
 14,
 121,
 12,
 3,
 1,
 14,
 6,
 8,
 5,
 2,
 4,
 85,
 17,
 1,
 1,
 6,
 1,
 1,
 7,
 102,
 17,
 31,
 13,
 8,
 5,
 67,
 49,
 23,
 119,
 5,
 14,
 10,
 5,
 2,
 5,
 1,
 1,
 11,
 1,
 8,
 1,
 8,
 8,
 8,
 8,
 8,
 8,
 32,
 16,
 17,
 8,
 11,
 9,
 8,
 177,
 248,
 5,
 2,
 29,
 7,
 20,
 2,
 2,
 2,
 2,
 3,
 5,
 7,
 2,
 2,
 7,
 14,
 93,
 8,
 3,
 71,
 2,
 7,
 24,
 9,
 9,
 2,
 7,
 11,
 2,
 5,
 12,
 27,
 36,
 5,
 2,
 2,
 14,
 6,
 4,
 18,
 31,
 24,
 12,
 22,
 2,
 1,
 2,
 50,
 4,
 1,
 60,
 19,
 4,
 9,
 1,
 1,
 8,
 33,
 41,
 9,
 2,
 4,
 11,
 12,
 4,
 10,

###### data = {'word': word_list, 'count' : word_rank}
word_data = pd.DataFrame.from_dict(data)
word_data

In [61]:
word_data = word_data.sort_values(by=['count'], ascending=False).reset_index().drop(columns=['index'])

In [62]:
word_data

,word,count
0,배송,248
1,새벽,177
2,마켓,129
3,물류,121
4,오아시스,119
...,...,...
2523,차량,1
2524,781,1
2525,5일,1
2526,200,1


In [28]:
for i in range(19):
    print(word_data[ i*10 : (i+1)*10 ])

   word  count
0    배송     11
1    새벽     10
2  오아시스      8
3    시장      5
4   IPO      5
5    마켓      4
6    물류      4
7    센터      4
8    경쟁      4
9   SSG      3
   word  count
10   컬리      3
11    재      3
12   생협      3
13  네이버      3
14    요      3
15   대표      3
16   내년      3
17    상      2
18   유통      2
19   장사      2
   word  count
20   루트      2
21    속      2
22   확대      2
23    기      2
24    백      2
25   출혈      2
26    강      2
27    나      2
28    등      2
29   펴다      2
    word  count
30    상장      2
31    간판      2
32    위상      2
33   플랫폼      2
34    재벌      2
35    총수      2
36    마트      2
37   장보기      2
38  치열하다      2
39    세기      2
    word  count
40    안정      2
41  평화시장      2
42    21      2
43   짜내다      2
44   노동자      2
45     불      2
46     1      2
47     호      2
48    누가      2
49    기업      2
   word  count
50   노동      2
51   핵심      2
52    개      1
53   부활      1
54    퀵      1
55  커머스      1
56    5      1
57  유니콘      1
58   1조      1
59 

In [64]:
word_data.to_csv('sentimental(news)_.csv')

In [69]:
sentimental = pd.read_csv('sentimental(news).csv').drop(columns=['Unnamed: 0'])
# sentimental = sentimental.fillna('-')
sentimental.insert(2,'label','-')

In [31]:
# sentimental.to_csv('sentimental(news).csv')

In [70]:
sentimental

,word,count,label
0,배송,248,-
1,새벽,177,-
2,마켓,129,-
3,물류,121,-
4,오아시스,119,-
...,...,...,...
2523,차량,1,-
2524,781,1,-
2525,5일,1,-
2526,200,1,-


### 추출했던 word_list에서 불용어 추가

In [ ]:
import csv

with open("stopword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(word_list)

In [ ]:
with open('stopword.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
stopword_add = sum(data, [])
stopword_add